In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

swara_dict = {
    "s": 0,
    "R1": 100,
    "R2": 200,
    "G1": 300,
    "G3": 400,
    "M1": 500,
    "M2": 600,
    "P": 700,
    "D1": 800,
    "D2": 900,
    "N2": 1000,
    "N3": 1100,
    "S": 1200
}

def find_closest_swara(mean_cents):
    """
    Given a mean in cents, find the closest swara

    Parameters:
    mean_cents (float): the mean in cents

    Returns:
    str: the closest swara
    """
    closest_swara = min(swara_dict, key=lambda k: abs(swara_dict[k] - mean_cents))
    return closest_swara

def append_df(dfs, raga, piece, section, instrument, file_path):
    """
    Append a dataframe to a list of dataframes

    Parameters:
    dfs (list): the list of dataframes
    raga (str): the raga
    piece (str): the piece
    section (str): the section
    instrument (str): the instrument
    file_path (str): the file path

    Returns:
    None
    """
    df = pd.read_csv(file_path)
    df["raga"] = raga
    df["piece"] = piece
    df["section"] = section
    df["instrument"] = instrument
    dfs.append(df)

In [2]:
# Load track info
tracks = [
    {"raga": "thodi", "piece": "Karuna Nidhi Illalo"},
    {"raga": "thodi", "piece": "Koluvamaregatha"},
    {"raga": "thodi", "piece": "Munnu Ravana"},
    {"raga": "thodi", "piece": "Thappi Bratikipova"},
    {"raga": "kalyani", "piece": "Kailasapathe"},
    {"raga": "kalyani", "piece": "Sundari Nee Divya"},
    {"raga": "kalyani", "piece": "Vandalum"}
]
# Initialize an empty list to store DataFrames
dfs = []

# Load data
for track in tracks:
    raga = track["raga"]
    piece = track["piece"]
    if piece == "Munnu Ravana" or piece == "Vandalum":
        vifile = f"{raga}/{piece}/{piece}.multitrack-violin-peaks.csv"
        append_df(dfs, raga, piece, "main", "violin", vifile)
        
        vofile = f"{raga}/{piece}/{piece}.multitrack-vocal-peaks.csv"
        append_df(dfs, raga, piece, "main", "vocal", vofile)
    else:
        viafile = f"{raga}/{piece}/{piece}.multitrack-violin-alaapana-peaks.csv"
        append_df(dfs, raga, piece, "alaapana", "violin", viafile)
        
        vimfile = f"{raga}/{piece}/{piece}.multitrack-violin-main-peaks.csv"
        append_df(dfs, raga, piece, "main", "violin", vimfile)
        
        visfile = f"{raga}/{piece}/{piece}.multitrack-violin-solo-peaks.csv"
        append_df(dfs, raga, piece, "solo", "violin", visfile)
        
        voafile = f"{raga}/{piece}/{piece}.multitrack-vocal-alaapana-peaks.csv"
        append_df(dfs, raga, piece, "alaapana", "vocal", voafile)
        
        vomfile = f"{raga}/{piece}/{piece}.multitrack-vocal-main-peaks.csv"
        append_df(dfs, raga, piece, "main", "vocal", vomfile)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Move piece, section, and instrument columns to the front
cols = combined_df.columns.tolist()
cols = cols[-4:] + cols[:-4]
combined_df = combined_df[cols]
        

In [5]:
# Find closest swara for each peak
combined_df['swara'] = combined_df['Mean (cents)'].apply(find_closest_swara)

# Move swara before mean cents
cols = combined_df.columns.tolist()
swara_col = cols.pop(cols.index('swara'))
cols.insert(cols.index('Mean (cents)'), swara_col)
combined_df = combined_df[cols]

# Add new column 'peak height' = (Amplitude (normalized) + Offset (normalized))/2
combined_df['peak height'] = (combined_df['Amplitude (normalized)'] + combined_df['Offset (normalized)'])/2

# Save combined DataFrame to a CSV file
combined_df.to_csv("swara_metrics.csv", index=False)

In [25]:
# Group by 'raga', 'instrument', and 'swara' and calculate the mean of the metrics
metrics_df = combined_df.groupby(['raga', 'instrument', 'swara']).agg({
    'Mean (cents)': 'mean',
    'Standard Deviation (cents)': 'mean',
    'Skewness': 'mean',
    'Amplitude (normalized)': 'mean'
}).reset_index()

# Filter for raga 'thodi' and instrument 'violin' or 'vocal'
thodi_metrics_df = metrics_df[(metrics_df['raga'] == 'thodi') & 
                              (metrics_df['instrument'].isin(['violin', 'vocal']))]

kalyani_metrics_df = metrics_df[(metrics_df['raga'] == 'kalyani') & 
                                (metrics_df['instrument'].isin(['violin', 'vocal']))]

In [26]:
thodi_metrics_df

,raga,instrument,swara,Mean (cents),Standard Deviation (cents),Skewness,Amplitude (normalized)
16,thodi,violin,D1,791.444444,19.338037,-1.497195,0.044693
17,thodi,violin,D2,891.000000,25.276667,3.654419,0.096042
18,thodi,violin,M1,494.400000,14.359682,-0.985879,0.156691
19,thodi,violin,N2,992.800000,20.383729,-1.427408,0.024380
20,thodi,violin,P,699.000000,10.870991,0.597376,0.192123
21,thodi,violin,R1,93.333333,50.260107,-2.314439,0.175854
22,thodi,violin,R2,194.750000,27.834569,4.034842,0.021813
23,thodi,violin,S,1194.500000,6.741302,1.057647,0.334178
24,thodi,violin,s,4.000000,3.691653,-1.472309,0.119120
25,thodi,vocal,D1,798.333333,24.479744,-0.876640,0.017441


In [35]:
# Create 4 DataFrames corresponding to 'thodi' and 'kalyani' ragas, 'violin' and 'vocal' instruments
thodi_violin = thodi_metrics_df[thodi_metrics_df['instrument'] == 'violin'].sort_values('swara', key=lambda x: x.map(swara_dict))
thodi_violin = thodi_violin.round({'Standard Deviation (cents)': 2, 'Skewness': 2, 'Amplitude (normalized)': 2})
thodi_violin['Mean (cents)'] = thodi_violin['Mean (cents)'].astype(int)

thodi_vocal = thodi_metrics_df[thodi_metrics_df['instrument'] == 'vocal'].sort_values('swara', key=lambda x: x.map(swara_dict))
thodi_vocal = thodi_vocal.round({'Standard Deviation (cents)': 2, 'Skewness': 2, 'Amplitude (normalized)': 2})
thodi_vocal['Mean (cents)'] = thodi_vocal['Mean (cents)'].astype(int)

kalyani_violin = kalyani_metrics_df[kalyani_metrics_df['instrument'] == 'violin'].sort_values('swara', key=lambda x: x.map(swara_dict))
kalyani_violin = kalyani_violin.round({'Standard Deviation (cents)': 2, 'Skewness': 2, 'Amplitude (normalized)': 2})
kalyani_violin['Mean (cents)'] = kalyani_violin['Mean (cents)'].astype(int)

kalyani_vocal = kalyani_metrics_df[kalyani_metrics_df['instrument'] == 'vocal'].sort_values('swara', key=lambda x: x.map(swara_dict))
kalyani_vocal = kalyani_vocal.round({'Standard Deviation (cents)': 2, 'Skewness': 2, 'Amplitude (normalized)': 2})
kalyani_vocal['Mean (cents)'] = kalyani_vocal['Mean (cents)'].astype(int)

In [39]:
kalyani_vocal

,raga,instrument,swara,Mean (cents),Standard Deviation (cents),Skewness,Amplitude (normalized)
15,kalyani,vocal,s,12,6.56,0.32,0.18
13,kalyani,vocal,R2,188,22.05,-0.12,0.10
9,kalyani,vocal,G3,374,36.33,0.74,0.28
10,kalyani,vocal,M2,607,19.45,1.48,0.04
12,kalyani,vocal,P,687,29.93,-3.30,0.25
8,kalyani,vocal,D2,888,30.68,2.74,0.06
11,kalyani,vocal,N3,1098,19.85,1.96,0.14
14,kalyani,vocal,S,1183,26.92,-4.33,0.29
